In [4]:
IF OBJECT_ID('Students', 'U') IS NOT NULL
    DROP TABLE Students;

CREATE TABLE Students
(
    id INT CONSTRAINT PK_UsersID PRIMARY KEY IDENTITY,
    name NVARCHAR(50) NOT NULL,
    subject NVARCHAR(50) NOT NULL,
    facility NVARCHAR(50) NOT NULL,
    grade REAL NOT NULL
);

Commands completed successfully.

Total execution time: 00:00:00.023

### Создание таблицы

In [5]:
INSERT INTO Students (name, subject, facility, grade)
VALUES
    (N'Евдокимова', N'математика', N'лицей', 98.5),
    (N'Подольский', N'физика', N'гимназия', 99),
    (N'Сидоров', N'математика', N'лицей', 88),
    (N'Петрова', N'физика', N'гимназия', 78),
    (N'Иванова', N'история', N'лицей', 92),
    (N'Кенджаев', N'история', N'гимназия', 68),
    (N'Иногороднев', N'математика', N'гимназия', 81),
    (N'Авдюшев', N'физика', N'лицей', 87),
    (N'Измайлова', N'история', N'лицей', 94),
    (N'Абдушев', N'история', N'лицей', 88);

(10 rows affected)

Total execution time: 00:00:00.014

### Заполнение данными

In [7]:
SELECT subject, facility, MAX(grade) AS StudentMaxGrade
FROM Students
GROUP BY subject, facility WITH ROLLUP;

(10 rows affected)

Total execution time: 00:00:00.005

subject,facility,StudentMaxGrade
история,гимназия,68
история,лицей,94
история,NULL,94
математика,гимназия,81
математика,лицей,98.5
математика,NULL,98.5
физика,гимназия,99
физика,лицей,87
физика,NULL,99
NULL,NULL,99


### Запрос, который выводит максимальный балл учеников по школам, по каждому предмету по каждой школе и промежуточные итоги

In [8]:
SELECT facility, subject, MIN(grade) AS StudentMinGrade 
FROM Students
GROUP BY facility, subject WITH CUBE;

(12 rows affected)

Total execution time: 00:00:00.016

facility,subject,StudentMinGrade
гимназия,история,68
лицей,история,88
NULL,история,68
гимназия,математика,81
лицей,математика,88
NULL,математика,81
гимназия,физика,78
лицей,физика,87
NULL,физика,78
NULL,NULL,68


### Запрос, который выводит минимальный балл учеников по школам и по предметам, и промежуточные итоги

In [9]:
SELECT facility, subject, AVG(grade) AS AverageScore
FROM Students
GROUP BY GROUPING SETS (facility, subject);

(5 rows affected)

Total execution time: 00:00:00.013

facility,subject,AverageScore
NULL,история,85.5
NULL,математика,89.16666666666667
NULL,физика,88
гимназия,NULL,81.5
лицей,NULL,91.25


### Запрос, который выводит средний балл учеников по школам и по предметам

In [11]:
Select 
	COALESCE (facility, 'Total Result') AS FacilityType,
	COALESCE (subject, 'Result') AS SubjectType,
	count (name) AS Amount
	from Students
	Group by 
		ROLLUP(facility, subject);

(9 rows affected)

Total execution time: 00:00:00.007

FacilityType,SubjectType,Amount
гимназия,история,1
гимназия,математика,1
гимназия,физика,2
гимназия,Result,4
лицей,история,3
лицей,математика,2
лицей,физика,1
лицей,Result,6
Total Result,Result,10


### Запрос, который выводит средний балл учеников по школам и по предметам

In [12]:
SELECT 
    IIF(Grouping(facility)=1, 'Total Result', facility) AS facility,
    IIF(Grouping(subject)=1, 'Result', subject) AS subject,
    SUM(grade) AS SummaryScore
FROM 
    Students
GROUP BY 
    CUBE(facility, subject);


(12 rows affected)

Total execution time: 00:00:00.007

facility,subject,SummaryScore
гимназия,история,68
лицей,история,274
Total Result,история,342
гимназия,математика,81
лицей,математика,186.5
Total Result,математика,267.5
гимназия,физика,177
лицей,физика,87
Total Result,физика,264
Total Result,Result,873.5


### Запрос, который выводит суммарный балл учеников по школам и по предметам, и промежуточные итоги. В итоговых строках NULL значения заменить на соответствующий текст в зависимости от группировки

In [13]:
SELECT 
	CASE GROUPING_ID(s.facility, s.subject)
		WHEN 1 THEN 'Results by school'
		WHEN 3 THEN 'Results'
		ELSE ''
	END AS [Предмет], 
	ISNULL(s.facility, '') AS [Школа],
	ISNULL(s.subject, '') AS [Предмет], 
	MAX(s.grade) AS [Максимальный балл]
FROM 
	Students s
GROUP BY 
	ROLLUP(s.facility, s.subject);


(9 rows affected)

Total execution time: 00:00:00.006

Предмет,Школа,Предмет,Максимальный балл
,гимназия,история,68
,гимназия,математика,81
,гимназия,физика,99
Results by school,гимназия,,99
,лицей,история,94
,лицей,математика,98.5
,лицей,физика,87
Results by school,лицей,,98.5
Results,,,99


### Запрос, который выводит максимальный балл учеников по школам и по предметам. В итоговых строках NULL значения заменить на соответствующий текст в зависимости от уровней группировки